# https://urlis.net/5mj36  (on Discord as well)

# Part0: Preparation


In [ ]:
# download data
_ = !wget http://www.robots.ox.ac.uk/~htd/misc/center_64x64_rgb.tar
_ = !wget http://www.robots.ox.ac.uk/~htd/misc/center_64x64_flow.tar
_ = !wget http://www.robots.ox.ac.uk/~htd/misc/train_split01.txt
_ = !wget http://www.robots.ox.ac.uk/~htd/misc/test_split01.txt
_ = !wget http://www.robots.ox.ac.uk/~htd/misc/ucf101_action.txt
_ = !wget http://www.robots.ox.ac.uk/~htd/misc/golf_swing.tar
_ = !wget http://www.robots.ox.ac.uk/~htd/misc/kitchen.tar

In [ ]:
_ = !tar xf center_64x64_rgb.tar
_ = !tar xf center_64x64_flow.tar
_ = !tar xf golf_swing.tar
_ = !tar xf kitchen.tar

In [ ]:
!ls

center_64x64_flow      golf_swing	 train_split01.txt
center_64x64_flow.tar  golf_swing.tar	 ucf101_action.txt
center_64x64_rgb       sample_data
center_64x64_rgb.tar   test_split01.txt


# Part1: Multi-modality in video: optical flow
Video naturally comes with multiple modalities (thinking of YouTube videos for example): RGB frames, audio, and text (video titles, tags, speech subtitles, comments). In this part we introduce a unique modality in video data: **optical flow**.

[Wikipedia](https://en.wikipedia.org/wiki/Optical_flow): Optical flow or optic flow is the pattern of apparent motion of objects, surfaces, and edges in a visual scene caused by the relative motion between an observer and a scene.

It is shown to be a complementary information with the RGB stream: [Two-Stream Convolutional Networks
for Action Recognition in Videos](https://proceedings.neurips.cc/paper/2014/file/00ec53c4682d36f5c4359f4ae7bd7ba1-Paper.pdf)

Let's take a closer look with examples!

In [ ]:
import cv2
import matplotlib.pyplot as plt

def compute_TVL1(prev, curr):
    """Compute the TV-L1 optical flow. https://docs.opencv.org/4.x/dc/d4d/classcv_1_1optflow_1_1DualTVL1OpticalFlow.html """
    TVL1 = cv2.optflow.DualTVL1OpticalFlow_create()
    flow = TVL1.calc(prev, curr, None)
    return flow

In [ ]:
!ls golf_swing/

1-00035.jpg  1-00037.jpg  2-00045.jpg  2-00047.jpg  3-00082.jpg  3-00084.jpg
1-00036.jpg  1-00038.jpg  2-00046.jpg  2-00048.jpg  3-00083.jpg  3-00085.jpg


In [ ]:
prev = cv2.imread('golf_swing/3-00082.jpg')
prev = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
curr = cv2.imread('golf_swing/3-00083.jpg')
curr = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
flow = compute_TVL1(prev, curr)

# Q1: what is the shape of the optical flow?
# Q2: how would you visualize it?

In [ ]:
# a recent visualization method: https://github.com/princeton-vl/RAFT/blob/master/core/utils/flow_viz.py 

# Part2: SSL preliminary

## 2.1. Import modules

In [ ]:
gpu_info = !nvidia-smi -i 0
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

from datetime import datetime
from functools import partial
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import resnet
from tqdm import tqdm
import argparse
import json
import math
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

Tue Jul 12 08:57:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 2.2. Set arguments

In [ ]:
parser = argparse.ArgumentParser(description='Train MoCo on CIFAR-10')

parser.add_argument('-a', '--arch', default='resnet18')

# lr: 0.06 for batch 512 (or 0.03 for batch 256)
parser.add_argument('--lr', '--learning-rate', default=0.06, type=float, metavar='LR', help='initial learning rate', dest='lr')
parser.add_argument('--epochs', default=10, type=int, metavar='N', help='number of total epochs to run')
parser.add_argument('--schedule', default=[120, 160], nargs='*', type=int, help='learning rate schedule (when to drop lr by 10x); does not take effect if --cos is on')
parser.add_argument('--cos', action='store_true', help='use cosine lr schedule')

parser.add_argument('--batch-size', default=256, type=int, metavar='N', help='mini-batch size')
parser.add_argument('--wd', default=5e-4, type=float, metavar='W', help='weight decay')

# moco specific configs:
parser.add_argument('--moco-dim', default=128, type=int, help='feature dimension')
parser.add_argument('--moco-k', default=4096, type=int, help='queue size; number of negative keys')
parser.add_argument('--moco-m', default=0.99, type=float, help='moco momentum of updating key encoder')
parser.add_argument('--moco-t', default=0.07, type=float, help='softmax temperature')

parser.add_argument('--bn-splits', default=8, type=int, help='simulate multi-gpu behavior of BatchNorm in one gpu; 1 is SyncBatchNorm in multi-gpu')

parser.add_argument('--symmetric', action='store_true', help='use a symmetric loss function that backprops to both crops')

# knn monitor
parser.add_argument('--knn-k', default=10, type=int, help='k in kNN monitor')
parser.add_argument('--knn-t', default=0.07, type=float, help='softmax temperature in kNN monitor; could be different with moco-t')

# utils
parser.add_argument('--resume', default='', type=str, metavar='PATH', help='path to latest checkpoint (default: none)')
parser.add_argument('--results-dir', default='', type=str, metavar='PATH', help='path to cache (default: none)')

'''
args = parser.parse_args()  # running in command line
'''
args = parser.parse_args('')  # running in ipynb

# set command line arguments here when running in ipynb
args.epochs = 200
args.cos = True
args.schedule = []  # cos in use
args.symmetric = False
if args.results_dir == '':
    args.results_dir = './cache-' + datetime.now().strftime("%Y-%m-%d-%H-%M-%S-ssl")

print(args)

Namespace(arch='resnet18', batch_size=256, bn_splits=8, cos=True, epochs=200, knn_k=10, knn_t=0.07, lr=0.06, moco_dim=128, moco_k=4096, moco_m=0.99, moco_t=0.07, results_dir='./cache-2022-07-12-09-26-21-ssl', resume='', schedule=[], symmetric=False, wd=0.0005)


## 2.3. Define data loaders

In [ ]:
def load_txt(path):
  with open(path) as fobj:
    content = fobj.readlines()
    return [i.rstrip() for i in content]

def pil_loader(path):
  with open(path, 'rb') as f:
    with Image.open(f) as img:
      return img.convert('RGB')


class MiniUCF101Pair():
    """Mini-version of UCF101 Dataset.
    """
    def __init__(self, root, train=False, transform=None):
      self.root = root
      self.train = train
      self.transform = transform
      if train:
        self.videolist = load_txt('train_split01.txt')
      else:
        self.videolist = load_txt('test_split01.txt')
      action_list = load_txt('ucf101_action.txt')
      self.act2idx = {act:i for i,act in enumerate(action_list)}

    def __len__(self):
      return len(self.videolist)

    def __getitem__(self, index):
        vname = self.videolist[index]
        img = pil_loader(os.path.join(self.root, f'{vname}.jpg'))

        if self.transform is not None:
            im_1 = self.transform(img)
            im_2 = self.transform(img)

        return im_1, im_2


class MiniUCF101():
    """Mini-version of UCF101 Dataset.
    """
    def __init__(self, root, train=False, transform=None):
      self.root = root
      self.train = train
      self.transform = transform
      if train:
        self.videolist = load_txt('train_split01.txt')
      else:
        self.videolist = load_txt('test_split01.txt')
      action_list = load_txt('ucf101_action.txt')
      self.act2idx = {act:i for i,act in enumerate(action_list)}

    def __len__(self):
      return len(self.videolist)

    def __getitem__(self, index):
        vname = self.videolist[index]
        img = pil_loader(os.path.join(self.root, f'{vname}.jpg'))

        if self.transform is not None:
            im = self.transform(img)

        label = self.act2idx[vname.split('/')[0]]
        return im, label


train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

# data prepare
train_data = MiniUCF101Pair(root='center_64x64_rgb', train=True, transform=train_transform)
train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)

memory_data = MiniUCF101(root='center_64x64_rgb', train=True, transform=test_transform)
memory_loader = DataLoader(memory_data, batch_size=args.batch_size, shuffle=False, num_workers=2, pin_memory=True)

test_data = MiniUCF101(root='center_64x64_rgb', train=False, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=args.batch_size, shuffle=False, num_workers=2, pin_memory=True)

## 2.3.1. (Optional): can you visualize these augmented images?

In [ ]:
vname = train_data.videolist[0]
img = pil_loader(os.path.join(train_data.root, f'{vname}.jpg'))
for _ in range(10):
  aug_img = train_transform(img)
  # visualize?

## 2.4. Define base encoder

In [ ]:
class ModelBase(nn.Module):
  """
  Common CIFAR ResNet recipe.
  Comparing with ImageNet ResNet recipe, it:
  (i) replaces conv1 with kernel=3, str=1
  (ii) removes pool1
  """
  def __init__(self, feature_dim=128, arch=None, bn_splits=16):
      super(ModelBase, self).__init__()

      # use split batchnorm
      # norm_layer = partial(SplitBatchNorm, num_splits=bn_splits) if bn_splits > 1 else nn.BatchNorm2d
      norm_layer = nn.BatchNorm2d
      resnet_arch = getattr(resnet, arch)
      net = resnet_arch(num_classes=feature_dim, norm_layer=norm_layer)

      self.net = []
      for name, module in net.named_children():
          if name == 'conv1':
              module = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
          if isinstance(module, nn.MaxPool2d):
              continue
          if isinstance(module, nn.Linear):
              self.net.append(nn.Flatten(1))
          self.net.append(module)

      self.net = nn.Sequential(*self.net)

  def forward(self, x):
      x = self.net(x)
      # note: not normalized here
      return x

## 2.5. Define info-NCE wrapper

<img src="http://www.robots.ox.ac.uk/~htd/misc/info-nce.png" width="400"/> 

Reference: [CPC](https://arxiv.org/abs/1807.03748), [SimCLR](https://arxiv.org/abs/2002.05709), [MoCo](https://arxiv.org/abs/1911.05722).

In [ ]:
class InfoNCE(ModelBase):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
  
  def forward(self, x1, x2):
    feature1 = self.net(x1)  # BxC
    feature2 = self.net(x2)  # BxC
    
    # define info-NCE loss here: you can use temperature 0.07
    loss = None
    
    return loss

model = InfoNCE(arch='resnet18')

## 2.6. Define train/test

In [ ]:
# train for one epoch
def train(net, data_loader, train_optimizer, epoch, args):
    net.train()
    adjust_learning_rate(optimizer, epoch, args)
    use_cuda = torch.cuda.is_available()

    total_loss, total_num, train_bar = 0.0, 0, tqdm(data_loader)
    for im_1, im_2 in train_bar:
      if use_cuda:
        im_1, im_2 = im_1.cuda(non_blocking=True), im_2.cuda(non_blocking=True)

      loss = net(im_1, im_2)
      train_optimizer.zero_grad()
      loss.backward()
      train_optimizer.step()

      total_num += data_loader.batch_size
      total_loss += loss.item() * data_loader.batch_size
      train_bar.set_description('Train Epoch: [{}/{}], lr: {:.6f}, Loss: {:.4f}'.format(epoch, args.epochs, optimizer.param_groups[0]['lr'], total_loss / total_num))

    return total_loss / total_num

# lr scheduler for training
def adjust_learning_rate(optimizer, epoch, args):
    """Decay the learning rate based on schedule"""
    lr = args.lr
    if args.cos:  # cosine lr schedule
        lr *= 0.5 * (1. + math.cos(math.pi * epoch / args.epochs))
    else:  # stepwise lr schedule
        for milestone in args.schedule:
            lr *= 0.1 if epoch >= milestone else 1.
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
# test using a knn monitor
def test(net, memory_data_loader, test_data_loader, epoch, args):
    net.eval()
    classes = len(memory_data_loader.dataset.act2idx)
    total_top1, total_top5, total_num, feature_bank = 0.0, 0.0, 0, []
    bank_label = []
    with torch.no_grad():
        # generate feature bank
        for data, target in tqdm(memory_data_loader, desc='Feature extracting'):
            feature = net(data.cuda(non_blocking=True))
            feature = F.normalize(feature, dim=1)
            feature_bank.append(feature)
            bank_label.append(target)
        # [D, N]
        feature_bank = torch.cat(feature_bank, dim=0).t().contiguous()
        feature_labels = torch.cat(bank_label, dim=0).to(feature_bank.device)

        # [N]
        # feature_labels = torch.tensor(memory_data_loader.dataset.targets, device=feature_bank.device)
        # loop test data to predict the label by weighted knn search
        test_bar = tqdm(test_data_loader)
        for data, target in test_bar:
            data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)
            feature = net(data)
            feature = F.normalize(feature, dim=1)
            
            pred_labels = knn_predict(feature, feature_bank, feature_labels, classes, args.knn_k, args.knn_t)

            total_num += data.size(0)
            total_top1 += (pred_labels[:, 0] == target).float().sum().item()
            test_bar.set_description('Test Epoch: [{}/{}] Acc@1:{:.2f}%'.format(epoch, args.epochs, total_top1 / total_num * 100))

    return total_top1 / total_num * 100


# knn monitor as in InstDisc https://arxiv.org/abs/1805.01978
# implementation follows http://github.com/zhirongw/lemniscate.pytorch and https://github.com/leftthomas/SimCLR
def knn_predict(feature, feature_bank, feature_labels, classes, knn_k, knn_t):
    # compute cos similarity between each feature vector and feature bank ---> [B, N]
    sim_matrix = torch.mm(feature, feature_bank)
    # [B, K]
    sim_weight, sim_indices = sim_matrix.topk(k=knn_k, dim=-1)
    # [B, K]
    sim_labels = torch.gather(feature_labels.expand(feature.size(0), -1), dim=-1, index=sim_indices)
    sim_weight = (sim_weight / knn_t).exp()

    # counts for each class
    one_hot_label = torch.zeros(feature.size(0) * knn_k, classes, device=sim_labels.device)
    # [B*K, C]
    one_hot_label = one_hot_label.scatter(dim=-1, index=sim_labels.view(-1, 1), value=1.0)
    # weighted score ---> [B, C]
    pred_scores = torch.sum(one_hot_label.view(feature.size(0), -1, classes) * sim_weight.unsqueeze(dim=-1), dim=1)

    pred_labels = pred_scores.argsort(dim=-1, descending=True)
    return pred_labels

## 2.7. Start training

In [ ]:
if torch.cuda.is_available():
  model.cuda()

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.wd, momentum=0.9)

# load model if resume
epoch_start = 1
if args.resume is not '':
    checkpoint = torch.load(args.resume)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch_start = checkpoint['epoch'] + 1
    print('Loaded from: {}'.format(args.resume))

# logging
results = {'train_loss': [], 'test_acc@1': []}
if not os.path.exists(args.results_dir):
    os.mkdir(args.results_dir)
# dump args
with open(args.results_dir + '/args.json', 'w') as fid:
    json.dump(args.__dict__, fid, indent=2)

# training loop
for epoch in range(epoch_start, args.epochs + 1):
    train_loss = train(model, train_loader, optimizer, epoch, args)
    results['train_loss'].append(train_loss)
    test_acc_1 = test(model.net, memory_loader, test_loader, epoch, args)
    results['test_acc@1'].append(test_acc_1)
    # save statistics
    data_frame = pd.DataFrame(data=results, index=range(epoch_start, epoch + 1))
    data_frame.to_csv(args.results_dir + '/log.csv', index_label='epoch')
    # save model
    torch.save({'epoch': epoch, 'state_dict': model.state_dict(), 'optimizer' : optimizer.state_dict(),}, args.results_dir + '/model_last.pth')

Test Epoch: [2/200] Acc@1:17.16%: 100%|██████████| 15/15 [00:01<00:00,  7.62it/s]
Train Epoch: [3/200], lr: 0.059967, Loss: 2.7908:  30%|██▉       | 11/37 [00:07<00:17,  1.47it/s]


KeyboardInterrupt: ignored

# Part3: Interesting findings on Optical flow

Recall your memory from Part1, do you think the optical flow from different golf-swing videos are similar?
How do we benefit from this property?

First, we can train an InfoNCE model on optical flow as well.

## 3.1. Prepare optical flow loader

In [ ]:
# data prepare
flow_train_data = MiniUCF101Pair(root='center_64x64_flow', train=True, transform=train_transform)
flow_train_loader = DataLoader(flow_train_data, batch_size=args.batch_size, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)

flow_memory_data = MiniUCF101(root='center_64x64_flow', train=True, transform=test_transform)
flow_memory_loader = DataLoader(flow_memory_data, batch_size=args.batch_size, shuffle=False, num_workers=2, pin_memory=True)

flow_test_data = MiniUCF101(root='center_64x64_flow', train=False, transform=test_transform)
flow_test_loader = DataLoader(flow_test_data, batch_size=args.batch_size, shuffle=False, num_workers=2, pin_memory=True)

## 3.2. Train the info-NCE for optical flow

In [ ]:
# define model
flow_model = InfoNCE(arch='resnet18')

# define optimizer
flow_optimizer = torch.optim.SGD(flow_model.parameters(), lr=args.lr, weight_decay=args.wd, momentum=0.9)

# load model if resume
epoch_start = 1
if args.resume is not '':
    checkpoint = torch.load(args.resume)
    flow_model.load_state_dict(checkpoint['state_dict'])
    flow_optimizer.load_state_dict(checkpoint['optimizer'])
    epoch_start = checkpoint['epoch'] + 1
    print('Loaded from: {}'.format(args.resume))

# logging
results = {'train_loss': [], 'test_acc@1': []}
if not os.path.exists(args.results_dir):
    os.mkdir(args.results_dir)
# dump args
with open(args.results_dir + '/args.json', 'w') as fid:
    json.dump(args.__dict__, fid, indent=2)

# training loop
for epoch in range(epoch_start, args.epochs + 1):
    train_loss = train(flow_model, flow_train_loader, flow_optimizer, epoch, args)
    results['train_loss'].append(train_loss)
    test_acc_1 = test(model.net, memory_loader, test_loader, epoch, args)
    results['test_acc@1'].append(test_acc_1)
    # save statistics
    data_frame = pd.DataFrame(data=results, index=range(epoch_start, epoch + 1))
    data_frame.to_csv(args.results_dir + '/log.csv', index_label='epoch')
    # save model
    torch.save({'epoch': epoch, 'state_dict': flow_model.state_dict(), 'optimizer' : flow_optimizer.state_dict(),}, args.results_dir + '/flow_model_last.pth')

# Part4: Self-supervised Co-Traininig

Inspired by our observations from the optical flow, we think we can use optical flow model to "teach" RGB model to learn better, and vice versa. Still, everything is self-supervised (no action label is used for training).

This is also the key idea of our NeurIPS2020 paper: 

[Self-supervised Co-training for Video Representation Learning](https://www.robots.ox.ac.uk/~vgg/research/CoCLR/)

<img src="https://www.robots.ox.ac.uk/~vgg/research/CoCLR/assets/images/coclr_teaser.png" width="500"/>

## 4.1. Prepare two-stream loader

In [ ]:
class MiniUCF101_2Stream_Pair():
    """Mini-version of UCF101 Dataset.
    """
    def __init__(self, root_rgb, root_flow, train=False, transform=None):
      self.root_rgb = root_rgb
      self.root_flow = root_flow
      self.train = train
      self.transform = transform
      if train:
        self.videolist = load_txt('train_split01.txt')
      else:
        self.videolist = load_txt('test_split01.txt')
      action_list = load_txt('ucf101_action.txt')
      self.act2idx = {act:i for i,act in enumerate(action_list)}

    def __len__(self):
      return len(self.videolist)

    def __getitem__(self, index):
        vname = self.videolist[index]
        img_rgb = pil_loader(os.path.join(self.root, f'{vname}.jpg'))
        img_flow = pil_loader(os.path.join(self.root, f'{vname}.jpg'))

        if self.transform is not None:
            im_1 = self.transform(img_rgb)
            im_2 = self.transform(img_rgb)

            flow_1 = self.transform(img_flow)
            flow_2 = self.transform(img_flow)

        return im_1, im_2, flow_1, flow_2

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

# data prepare
train_data_2s = MiniUCF101_2Stream_Pair(root_rgb='center_64x64_rgb', root_flow='center_64x64_flow', train=True, transform=train_transform)
train_loader_2s = DataLoader(train_data_2s, batch_size=args.batch_size, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)

test_data_2s = MiniUCF101_2Stream_Pair(root='center_64x64_rgb', root_flow='center_64x64_flow', train=False, transform=test_transform)
test_loader_2s = DataLoader(test_data_2s, batch_size=args.batch_size, shuffle=False, num_workers=2, pin_memory=True)

## 4.2. Define Co-Training wrapper

In [ ]:
class CoTraining(nn.Module):
  def __init__(self, *args, **kwargs):
    super().__init__()
    self.rgb_net = ModelBase(*args, **kwargs)
    self.flow_net = ModelBase(*args, **kwargs)
  
  def forward(self, x1, x2, f1, f2):
    rgb_feature1 = self.rgb_net.net(x1)  # BxC
    rgb_feature2 = self.rgb_net.net(x2)  # BxC
    flow_feature1 = self.flow_net.net(f1)  # BxC
    flow_feature2 = self.flow_net.net(f2)  # BxC
        
    rgb_sim = F.normalize(rgb_feature1, dim=1).matmul(F.normalize(rgb_feature2, dim=1).T).div(0.07)
    flow_sim = F.normalize(flow_feature1, dim=1).matmul(F.normalize(flow_feature2, dim=1).T).div(0.07)
    
    # define info-NCE loss here
    loss = None

    return loss

model = CoTraining(arch='resnet18')

## 4.3. (Optional) Co-Train rgb and flow networks

In [ ]:
# define model
cotrain_model = CoTraining(arch='resnet18')

# define optimizer
cotrain_optimizer = torch.optim.SGD(cotrain_model.parameters(), lr=args.lr, weight_decay=args.wd, momentum=0.9)

# load model if resume: 
epoch_start = 1
rgb_ckpt = ''
flow_ckpt = ''

rgb_checkpoint = torch.load(rgb_ckpt)['state_dict']
flow_checkpoint = torch.load(flow_ckpt)['state_dict']
combined_ckpt = {}
for k,v in rgb_checkpoint.items():
  combined_ckpt[f'rgb_net.{k}'] = v
for k,v in flow_checkpoint.items():
  combined_ckpt[f'flow_net.{k}'] = v
cotrain_model.load_state_dict(combined_ckpt)
print('Loaded from: {} and {}'.format(rgb_ckpt, flow_ckpt))

# logging
results = {'train_loss': [], 'test_acc@1': []}
if not os.path.exists(args.results_dir):
    os.mkdir(args.results_dir)
# dump args
with open(args.results_dir + '/args.json', 'w') as fid:
    json.dump(args.__dict__, fid, indent=2)

# training loop
for epoch in range(epoch_start, args.epochs + 1):
    train_loss = train(cotrain_model, train_loader_2s, cotrain_optimizer, epoch, args)
    results['train_loss'].append(train_loss)
    test_acc_1 = test(model.rgb_net.net, memory_loader, test_loader, epoch, args)
    results['test_acc@1'].append(test_acc_1)
    # save statistics
    data_frame = pd.DataFrame(data=results, index=range(epoch_start, epoch + 1))
    data_frame.to_csv(args.results_dir + '/log.csv', index_label='epoch')
    # save model
    torch.save({'epoch': epoch, 'state_dict': flow_model.state_dict(), 'optimizer' : flow_optimizer.state_dict(),}, args.results_dir + '/cotrain_model_last.pth')

## 4.4. Our paper and related works.

More details in our [paper](https://www.robots.ox.ac.uk/~vgg/research/CoCLR/):
* Memory bank to increase batch size
* Iteratively optimize two networks
* Pre-train on larger datasets

Other related works in this line of research: [SupportSet](https://arxiv.org/abs/2010.02824), [SwAV](https://arxiv.org/abs/2006.09882)


# Conclusion and Limitation

* Familar with optical flow in videos
* Understand classic SSL algorithm
* Multi-modal SSL algorithm
* Finally, an informal introduction of multi-modal SSL research now: [link](https://twitter.com/osanseviero/status/1546224159553986561?s=20&t=rYbQ2MWKaAPiZ13MUTxPBQ)

<img src="http://www.robots.ox.ac.uk/~htd/misc/gpu_figure.png" width="200"/>
